<a href="https://colab.research.google.com/github/clementsiegrist/segmentation_probabilisticDL/blob/main/bayesiannet_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras opencv-python tensorflow scikit-learn scikit-image numpy pillow matplotlib imutils

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

In [ ]:
https://github.com/biomedia-mira/stochastic_segmentation_network **** Données Cerveaux ****
To download the data, go to this page and follow the instructions provided by the challenge's organisers. Run the following script to preprocess the data (input-dir should contain folders HGG and LGG inside):
# python evaluation/preprocessing.py --input-dir <path-to-input-dir> --output-dir <path-to-output-dir>

https://github.com/abhinavsagar/uqvi **** DONNEE CERVAUX ****
Uncertainty Quantification using Variational Inference for Biomedical Image Segmentation
The dataset can be downloaded from here.

https://github.com/sandialabs/bcnn ****** NE DONNE PAS SES DONNEES Mais liobre utilisation ****

#### Cellule ####  Ihab exploite jeu de données cellules / Kaggle 
https://www.kaggle.com/rexhaif/morphological-postprocessing-on-unet-lb-0-429 

# Preprocessing the dataset Kaggle 
https://www.kaggle.com/dingli/keras-u-net-for-nuclei-segmentation 

https://github.com/xiaoketongxue/AI-News/blob/master/CV2019.md 



https://github.com/biomedia-mira/stochastic_segmentation_network
To download the data, go to this page and follow the instructions provided by the challenge's organisers. Run the following script to preprocess the data (input-dir should contain folders HGG and LGG inside):
# python evaluation/preprocessing.py --input-dir <path-to-input-dir> --output-dir <path-to-output-dir>

https://github.com/abhinavsagar/uqvi 
Uncertainty Quantification using Variational Inference for Biomedical Image Segmentation
The dataset can be downloaded from here.